# Parameters

The next code block sets parameters that are used throughout the remainder of the notebook.

In [ ]:
# pylint: disable=invalid-name,missing-module-docstring

# an integer, increment if you need to redo your analysis
# will be appended to your username to create analysis_id
analysis_number = 0

# experiment ID that must match the parent folder containing the LCMS output files
# An example experiment ID is '20201116_JGI-AK_LH_506489_SoilWarm_final_QE-HF_HILICZ_USHXG01530'
experiment = "REPLACE ME"

# group will only be used in RT prediction if their name has a substring match to this list of strings
include_groups = ["_QC_"]

# Exclude files with names containing any of the substrings in this list. Eg., ['peas', 'beans']
exclude_files = []

# list of substrings that will group together when creating groups
# this provides additional grouping beyond the default grouping on field #12
groups_controlled_vocab = ["QC", "InjBl", "ISTD"]

# thresholds for filtering out compounds with weak MS1 signals
num_points = 5
peak_height = 4e5

# The rest of this block contains project independent parameters

# to use an older version of the metatlas source code, set this to a commit id,
# branch name, or tag. If None, then use the the "main" branch.
source_code_version_id = None

# Full path to the directory where you want this notebook to store data.
# A subdirectory will be auto created within this directory for each project.
# You can place this anywhere on cori's filesystem, but placing it within your
# global home directory is recommended so that you do not need to worry about
# your data being purged. Each project will take on the order of 100 MB.
project_directory = "/global/homes/FIRST-INITIAL-OF-USERNAME/USERNAME/metabolomics_projects"

# ID from Google Drive URL for base output folder .
# The default value is the ID that corresponds to 'JGI_Metabolomics_Projects'.
google_folder = "0B-ZDcHbPi-aqZzE5V3hOZFc0dms"

# maximum number of CPUs to use
# when running on jupyter.nersc.gov, you are not allowed to set this above 4
max_cpus = 4

# Threshold for how much status information metatlas functions print in the notebook
# levels are 'DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL'
log_level = "INFO"

In [ ]:
# pylint: disable=wrong-import-position,import-error,missing-class-docstring
import logging  # noqa: E402
import os  # noqa: E402
from pathlib import Path  # noqa: E402


class StopExecution(Exception):
    def _render_traceback_(self):
        pass


logger = logging.getLogger("metatlas.jupyter")
kernel_def = """{"argv":["shifter","--entrypoint","--image=wjhjgi/metatlas_shifter:latest","/usr/local/bin/python","-m",
                 "ipykernel_launcher","-f","{connection_file}"],"display_name": "Metatlas Targeted","language": "python"}"""
kernel_file_name = Path.home() / ".local" / "share" / "jupyter" / "kernels" / "metatlas-targeted" / "kernel.json"
kernel_file_name.parent.mkdir(parents=True, exist_ok=True)
try:
    with kernel_file_name.open(mode="r", encoding="utf-8") as f:
        current_kernel_def = f.read()
except FileNotFoundError:
    current_kernel_def = ""
if "METATLAS_TEST" not in os.environ and current_kernel_def != kernel_def:
    with kernel_file_name.open(mode="w", encoding="utf-8") as f:
        f.writelines(kernel_def)
    logger.critical("CRITICAL: Notebook kernel has been updated. Please restart kernel and re-run notebook.")
    raise StopExecution
try:
    from metatlas.tools import notebook, predict_rt  # noqa: E402
except ImportError as err:
    logger.critical('CRITICAL: Please check that the kernel is set to "Metatlas Targeted".')
    raise StopExecution from err

notebook.setup(log_level, source_code_version_id)

In [ ]:
ids = predict_rt.get_analysis_ids_for_rt_prediction(experiment, project_directory, google_folder, analysis_number)

In [ ]:
predict_rt.generate_outputs(ids, max_cpus, num_points, peak_height)